## twitter data bigquery upload

In [ ]:
import os
import logging
import pprint
import json
import datetime
import pandas as pd 
import numpy as np 
import google.auth
from google.oauth2 import service_account
from google.cloud import bigquery
from google.cloud import bigquery_storage
print('imported modules successfully.')

### read .csv to upload

In [106]:
print(os.getcwd())
master = pd.read_csv('master_data_clean.csv')
master

/media/steven/big_boi/twitter


,text,user,date,fav_count,retweet_count,retweet
0,tb to when i was younger and my dad tried to p...,xanax_princess_,2020-05-24 23:48:11,5,0,False
1,"never thought I’d wear jeans in 90+ weather, b...",xanax_princess_,2020-05-19 23:01:57,3,0,False
2,new mixtape “i smoked 100 blunts with god” com...,xanax_princess_,2020-05-09 21:50:16,2,0,False
3,@annabanana4128 Ewwww gross,xanax_princess_,2020-05-08 00:15:20,1,0,False
4,@slpyboy @audiotreemusic i love diy,xanax_princess_,2020-05-07 19:13:14,1,0,False
...,...,...,...,...,...,...
13449,@cotty769 corn,Bean_glitch,2013-11-23 00:12:59,1,1,False
13450,"Just relapsed on coffee, so proud of myself.",Bean_glitch,2013-11-21 17:29:07,0,0,False
13451,I want to throw a rake at my professor. I have...,Bean_glitch,2013-11-21 16:10:05,0,0,False
13452,The best kind of hashtags are the hashtags wit...,Bean_glitch,2013-11-20 18:40:30,0,0,False


### set application credentials

In [72]:
creds_fname = '/media/steven/big_boi/test-e06ee679f03b.json'
client = bigquery.Client.from_service_account_json(creds_fname)
bqstorageclient = bigquery_storage.BigQueryStorageClient.from_service_account_json(creds_fname)
logging.info('initialized bigquery client.')

### get current table

In [105]:
QUERY = "SELECT * FROM `symbolic-bit-277217.basement_dude_tweets.tweets_master`;"
query_job = client.query(QUERY)
rows = query_job.result()

results = client.query(QUERY).result()

bq_output = results.to_dataframe()

In [107]:
bq_output

,text,user,date,fav_count,retweet_count,retweet
0,RT @j4mmar: https://t.co/EGOaULHisQ,slpyboy,2020-05-20 00:34:43+00:00,0,28,False
1,RT @j4mmar: https://t.co/EGOaULHisQ,slpyboy,2020-05-20 00:34:43+00:00,0,28,False
2,RT @ColumbiaRecords: YOGA // 645AR\nhttps://t....,slpyboy,2020-05-05 15:44:01+00:00,0,845,False
3,RT @willforthrill: Tom DeLonge every time UFOs...,slpyboy,2020-04-28 08:45:29+00:00,0,245,False
4,RT @lib_owner: less than a year apart https://...,slpyboy,2020-04-18 17:41:16+00:00,0,5013,False
...,...,...,...,...,...,...
9595,i only vape essential oils,xanax_princess_,2018-03-09 16:56:08+00:00,11,2,False
9596,https://t.co/y3DwvhzHGw\n\nmaybe my fav releas...,slpyboy,2018-04-30 14:20:35+00:00,11,2,False
9597,I presented a poster on my experience with @LI...,xanax_princess_,2015-04-24 18:22:13+00:00,12,2,False
9598,"That's like saying, ""I'm worried Gordon Ramsey...",cd3lisi,2019-10-07 15:43:03+00:00,16,2,False


### get new data to upload

remove date column b/c of difference b/t bigquery and dataframe date types.

In [117]:
to_upload = pd.concat([master.drop(columns = 'date'), bq_output.drop(columns = 'date')]).drop_duplicates(keep = False)

add date column back in from master .csv.

In [119]:
to_upload = pd.concat([to_upload, master.date], axis = 1, join = 'inner')
to_upload.reset_index(drop = True)

,text,user,fav_count,retweet_count,retweet,date
0,yeah this is basically how i make music https:...,xanax_princess_,2,0,False,2018-10-06 23:23:34
1,@pck____ sweet yeah pillow came up when i was ...,xanax_princess_,1,0,False,2018-09-02 19:58:03
2,my fav part about being a teacher is telling k...,xanax_princess_,5,0,False,2018-08-27 23:52:35
3,but the good news is some of the kids are nerd...,xanax_princess_,6,0,False,2018-08-18 01:24:27
4,lmaoooo day two of teaching and the kids alrea...,xanax_princess_,8,0,False,2018-08-18 00:12:46
...,...,...,...,...,...,...
3911,but the good news is some of the kids are nerd...,xanax_princess_,5,0,False,2018-06-08 20:29:09
3912,"""yeah we're going to the hippie capital of nor...",xanax_princess_,5,0,False,2018-06-08 12:53:33
3913,i used to crave drugs &amp; liquor on the week...,xanax_princess_,6,0,False,2018-04-06 04:39:32
3914,lmaoooo day two of teaching and the kids alrea...,xanax_princess_,7,0,False,2018-02-21 19:52:21


### upload dataframe to bigquery